In [ ]:
import requests
import pandas as pd

# --- Configuração ---
# Cole sua chave de API aqui (lembre-se de regerá-la se tiver mais de 24h)
API_KEY = "RGAPI-df9fd37f-7228-4ff6-8711-5af71626f14e" 

# O ID da partida que você quer examinar (exemplo)
MATCH_ID = "BR1_3118673257"

# A rota de plataforma correta para a partida (br1, euw1, na1, etc.)
SERVIDOR_JOGO = "br1"
# --------------------

headers = {
    "X-Riot-Token": API_KEY
}

def extrair_aprimoramentos_da_timeline(timeline_json, participantes_puuids):
    """
    Processa o JSON da timeline para extrair os aprimoramentos escolhidos por cada jogador.
    """
    # Mapeia o participantId (ex: 1 a 8) para o puuid
    id_para_puuid = {p['participantId']: p['puuid'] for p in participantes_puuids}
    
    # Dicionário para guardar os aprimoramentos de cada jogador
    aprimoramentos_por_jogador = {puuid: [] for puuid in id_para_puuid.values()}
    
    # Percorre todos os eventos da timeline
    for evento in timeline_json.get('info', {}).get('frames', []):
        for evento_jogador in evento.get('events', []):
            # Procuramos pelo evento específico de escolha de aprimoramento
            if evento_jogador.get('type') == 'TFT_AUGMENT_CHOSEN':
                participant_id = evento_jogador.get('participantId')
                augment_name = evento_jogador.get('augmentName')
                
                if participant_id and augment_name:
                    puuid_jogador = id_para_puuid.get(participant_id)
                    if puuid_jogador:
                        aprimoramentos_por_jogador[puuid_jogador].append(augment_name)
                        
    return aprimoramentos_por_jogador

# --- 1. Obter os detalhes da partida (para pegar a lista de participantes) ---
url_match_details = f"https://{SERVIDOR_JOGO}.api.riotgames.com/tft/match/v1/matches/{MATCH_ID}"
response_details = requests.get(url_match_details, headers=headers)

if response_details.status_code != 200:
    print(f"❌ Erro Crítico ao buscar detalhes da partida: {response_details.status_code}")
    print("   Não é possível continuar sem a lista de participantes.")
    print("   Causa provável: Sua chave de API é inválida ou não foi propagada para este servidor.")
    exit()

dados_partida = response_details.json()
participantes_info = dados_partida.get('info', {}).get('participants', [])
print(f"✅ Detalhes da partida {MATCH_ID} obtidos. Encontrados {len(participantes_info)} participantes.")


# --- 2. Obter a TIMELINE da partida ---
url_timeline = f"https://{SERVIDOR_JOGO}.api.riotgames.com/tft/match/v1/matches/{MATCH_ID}/timeline"
print("\nBuscando dados da timeline...")
response_timeline = requests.get(url_timeline, headers=headers)

if response_timeline.status_code == 200:
    print("✅ DADOS DA TIMELINE OBTIDOS COM SUCESSO!")
    dados_timeline = response_timeline.json()
    
    # --- 3. Processar a timeline e exibir os resultados ---
    aprimoramentos = extrair_aprimoramentos_da_timeline(dados_timeline, participantes_info)
    
    print("\n--- Aprimoramentos Escolhidos na Partida ---")
    
    # Criamos um DataFrame do pandas para uma exibição bonita
    # Primeiro, preparamos os dados
    display_data = []
    for p_info in participantes_info:
        puuid = p_info['puuid']
        placement = p_info['placement']
        augments = aprimoramentos.get(puuid, [])
        display_data.append({
            'Colocação': placement,
            'Aprimoramento_1': augments[0] if len(augments) > 0 else 'N/A',
            'Aprimoramento_2': augments[1] if len(augments) > 1 else 'N/A',
            'Aprimoramento_3': augments[2] if len(augments) > 2 else 'N/A',
            'PUUID': puuid
        })
    
    resultado_df = pd.DataFrame(display_data).sort_values(by='Colocação')
    print(resultado_df.to_string(index=False))

else:
    print(f"❌ Erro ao buscar a timeline: {response_timeline.status_code} - {response_timeline.text}")

❌ Erro Crítico ao buscar detalhes da partida: 403
   Não é possível continuar sem a lista de participantes.
   Causa provável: Sua chave de API é inválida ou não foi propagada para este servidor.
✅ Detalhes da partida BR1_3118673257 obtidos. Encontrados 0 participantes.

Buscando dados da timeline...
❌ Erro ao buscar a timeline: 403 - {"status":{"message":"Forbidden","status_code":403}}


: 